In [ ]:
# ========================================
# 1) Connect to Google Drive
# ========================================
#from google.colab import drive
#drive.mount('/content/drive')

#MODEL_DIR = "/content/drive/MyDrive/deep_learning"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ========================================
# 2) Install required libraries
# ========================================
!pip install -U transformers datasets accelerate evaluate optuna wandb


In [ ]:
!pip install evaluate

In [ ]:
!pip install -U transformers


In [ ]:
# ========================================
# 3) Import libraries
# ========================================
import os
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from collections import Counter
from pathlib import Path
import torch
from torch import nn

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import evaluate
import wandb
import types
import torch

In [ ]:
# Check if GPU is available and select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# ========================================
# 4) Setup Weights & Biases logging
# ========================================
os.environ["WANDB_PROJECT"] = "covid-HF-YS1"
os.environ["WANDB_WATCH"] = "all"
os.environ["WANDB_LOG_MODEL"] = "true"
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yardenshalom (yardenshalom-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:

# ========================================
# 3) Load data (your preprocessed CSVs)
# ========================================
#train_df = pd.read_csv('/content/drive/MyDrive/deep_learning/train_processed.csv', encoding='latin1')
#eval_df  = pd.read_csv('/content/drive/MyDrive/deep_learning/val_processed.csv',   encoding='latin1')
#test_df  = pd.read_csv('/content/drive/MyDrive/deep_learning/test_processed.csv',  encoding='latin1')


MODEL_DIR =Path.cwd()  # models/bert-base-uncased/
# Navigate from model folder to data folder
current_dir = Path.cwd()  # models/bert-base-uncased/
models_dir = current_dir.parent  # models/
project_root = models_dir.parent  # project root
data_dir = models_dir / 'data'

# Load data
train_df = pd.read_csv(data_dir / 'train_processed.csv', encoding='latin1')
eval_df = pd.read_csv(data_dir / 'val_processed.csv', encoding='latin1')
test_df = pd.read_csv(data_dir / 'test_processed.csv', encoding='latin1')

print(f"Data loaded from: {data_dir}")
print(f"Train shape: {train_df.shape}")
print(f"Val shape: {eval_df.shape}")
print(f"Test shape: {test_df.shape}")

# Create label mappings
ordered_labels = ['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
label2id = {label: i for i, label in enumerate(ordered_labels)}
id2label = {i: label for label, i in label2id.items()}

train_df["label"] = train_df["Sentiment"].map(label2id)
eval_df["label"] = eval_df["Sentiment"].map(label2id)
test_df["label"] = test_df["Sentiment"].map(label2id)

In [ ]:
# ========================================
# 4) Build HF Datasets and tokenize
# ========================================

model_ckpt = "bert-base-uncased"                 # <-- BERT cased
tokenizer  = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

specials = {"additional_special_tokens": ["<httpurl>", "<user>", "<hashtag>", "<emoji>"]}
tokenizer.add_special_tokens(specials)

def tok(batch):
    return tokenizer(
        batch["ProcessedTweet"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df[["ProcessedTweet","label"]].rename(columns={"label":"labels"})),
    "validation": Dataset.from_pandas(eval_df[["ProcessedTweet","label"]].rename(columns={"label":"labels"})),
    "test": Dataset.from_pandas(test_df[["ProcessedTweet","label"]].rename(columns={"label":"labels"})),
})

ds_tok = ds.map(tok, batched=True)
ds_tok = ds_tok.remove_columns(["ProcessedTweet"])
ds_tok.set_format("torch")


Map:   0%|          | 0/32925 [00:00<?, ? examples/s]

Map:   0%|          | 0/8232 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [ ]:
# ========================================
# 5) Optional: class weights (to handle class imbalance)
# ========================================
def compute_class_weights(int_labels, num_labels):
    """
    Compute inverse-frequency class weights normalized around 1.0
    """
    import numpy as np
    counts = np.bincount(int_labels, minlength=num_labels)
    weights = (counts.sum() / (counts + 1e-9)) / num_labels
    return weights / weights.mean()

class_weights = compute_class_weights(
    train_df["label"].to_numpy(),
    num_labels=len(ordered_labels)
)
class_weights
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
# ========================================
# 6) Custom BERT with configurable dropout (CLS pooling)
# ========================================
import torch.nn as nn
from transformers import AutoModel

class BertWithDropout(nn.Module):
    """
    Custom BERT classification head:
      - Takes the [CLS] token embedding (token at position 0)
      - Applies configurable dropout
      - Passes through a linear layer to produce logits for num_labels
      Works with 'bert-base-uncased' or any BERT-based model via AutoModel.
    """
    def __init__(self, model_name: str, num_labels: int, dropout_rate: float = 0.2):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(model_name)
        self.dropout  = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        cls = outputs.last_hidden_state[:, 0, :]
        x = self.dropout(cls)
        logits = self.classifier(x)
        return {"logits": logits}


In [ ]:
# ========================================
# 7) Weighted loss wrapper for Trainer
# ========================================
import numpy as np
import torch
import torch.nn as nn

ce_loss = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float))

def custom_compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    labels  = inputs.get("labels")
    outputs = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs.get("attention_mask")
    )
    logits = outputs["logits"] if isinstance(outputs, dict) else outputs.logits
    ce = nn.CrossEntropyLoss(weight=class_weights_tensor.to(logits.device))

    loss = ce(logits, labels)
    return (loss, outputs) if return_outputs else loss



In [ ]:
# ========================================
# 8) Metrics (accuracy + F1 micro/macro/weighted)
# ========================================
import evaluate
acc = evaluate.load("accuracy")
f1  = evaluate.load("f1")

def compute_metrics_fn(eval_pred):
    """
    HF Trainer metrics: returns dict with accuracy and F1 variants
    """
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":      acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_micro":      f1.compute(predictions=preds, references=labels, average="micro")["f1"],
        "f1_macro":      f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted":   f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }


In [ ]:
# ========================================
# 9) Trainer factory (no HF checkpoints) + BestWeightsSaver callback
#    - Saves ONLY best weights (state_dict) per run to a temp .pt
#    - EarlyStopping relies on eval_f1_weighted from compute_metrics_fn
# ========================================
import os
import gc
import torch
import types
from datetime import datetime
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback

class BestWeightsSaver(TrainerCallback):
    """
    Whenever 'eval_f1_weighted' improves, save ONLY model.state_dict() to `best_weights_path`.
    """
    def __init__(self, best_weights_path: str, metric_name: str = "eval_f1_weighted"):
        self.best_weights_path = best_weights_path
        self.metric_name = metric_name
        self.best_score = None
        self._trainer = None  # injected on attach

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if not metrics or self.metric_name not in metrics:
            return
        score = float(metrics[self.metric_name])
        if (self.best_score is None) or (score > self.best_score):
            self.best_score = score
            torch.save(self._trainer.model.state_dict(), self.best_weights_path)

MODEL_NAME = "bert-base-uncased"


def make_trainer(
    output_dir,
    dropout_rate=0.2,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    gradient_accumulation_steps=1,
    label_smoothing_factor=0.0,
    fp16=True,
    report_to_wandb=False,
    run_name="trial",
    best_weights_path="/tmp/best_weights.pt",
):
    # Build model with your custom head
    model = BertWithDropout(
        model_name=MODEL_NAME,
        num_labels=len(ordered_labels),
        dropout_rate=dropout_rate
    )

    # Absolutely NO HF checkpoints to Drive
    args = TrainingArguments(
        output_dir=output_dir,                 # keep this under /tmp to avoid Drive writes
        eval_strategy="epoch",
        save_strategy="no",                    # no checkpoints
        load_best_model_at_end=False,          # we handle "best" ourselves
        metric_for_best_model="f1_weighted",   # required for EarlyStopping
        greater_is_better=True,
        logging_strategy="steps",
        logging_steps=50,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        num_train_epochs=num_train_epochs,
        warmup_ratio=warmup_ratio,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        lr_scheduler_type=lr_scheduler_type,
        gradient_accumulation_steps=gradient_accumulation_steps,
        label_smoothing_factor=label_smoothing_factor,
        max_grad_norm=1.0,
        fp16=fp16,
        report_to=(["wandb"] if report_to_wandb else ["none"]),
        run_name=run_name,
        seed=42,
        dataloader_num_workers=2,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tok["train"],
        eval_dataset=ds_tok["validation"],
        processing_class=tokenizer,           # future-proof vs tokenizer=
        compute_metrics=compute_metrics_fn,
    )

    # Custom weighted CE loss (device-aware)
    trainer.compute_loss = types.MethodType(custom_compute_loss, trainer)

    # Early stopping (small patience to keep it agile)
    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

    # Attach best-weights saver
    best_cb = BestWeightsSaver(best_weights_path=best_weights_path, metric_name="eval_f1_weighted")
    best_cb._trainer = trainer
    trainer.add_callback(best_cb)

    return trainer


In [ ]:
# ========================================
# 10) Generic experiment runner (replaces your old "part 10")
#     - tune_once(): Optuna tuning for a given search space
#     - final_train_and_save(): long-ish final run + save ONLY weights to Drive
# ========================================
import json
import optuna
import pandas as pd

MODEL_DIR  =data_dir
EXPERIMENTS_LOG = os.path.join(MODEL_DIR, "HF_experiments_log_3.csv")  # append-only CSV

def now_tag():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def temp_paths(tag: str):
    stamp = now_tag()
    out_dir = f"/tmp/{tag}_{stamp}"
    best_pt = f"/tmp/{tag}_{stamp}_best.pt"
    os.makedirs(out_dir, exist_ok=True)
    return out_dir, best_pt

def append_row_to_log(row: dict):
    if os.path.exists(EXPERIMENTS_LOG):
        df = pd.read_csv(EXPERIMENTS_LOG)
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    else:
        df = pd.DataFrame([row])
    df.to_csv(EXPERIMENTS_LOG, index=False)

def tune_once(config_name: str, search_space_fn, n_trials=10, tune_epochs=(3,6)):
    """
    Run Optuna on a provided search space function.
    Returns (best_params, best_value).
    """
    def objective(trial: optuna.trial.Trial):
        hp = search_space_fn(trial, tune_epochs)
        out_dir, best_pt = temp_paths(f"tune_{config_name}_t{trial.number}")
        trainer = make_trainer(
            output_dir=out_dir,
            dropout_rate=hp["dropout_rate"],
            learning_rate=hp["learning_rate"],
            weight_decay=hp["weight_decay"],
            warmup_ratio=hp["warmup_ratio"],
            lr_scheduler_type=hp["lr_scheduler_type"],
            per_device_train_batch_size=hp["per_device_train_batch_size"],
            per_device_eval_batch_size=64,
            num_train_epochs=hp["num_train_epochs"],
            gradient_accumulation_steps=hp["gradient_accumulation_steps"],
            label_smoothing_factor=hp["label_smoothing_factor"],
            fp16=True,
            report_to_wandb=False,
            run_name=f"{config_name}-trial-{trial.number}",
            best_weights_path=best_pt
        )
        trainer.train()
        metrics = trainer.evaluate(ds_tok["validation"])
        score = float(metrics.get("eval_f1_weighted") or metrics.get("f1_weighted") or 0.0)

        # Clean temp
        try:
            if os.path.exists(best_pt):
                os.remove(best_pt)
        except OSError:
            pass
        del trainer
        torch.cuda.empty_cache(); gc.collect()
        return score

    study_name = f"{config_name}_{now_tag()}"
    study = optuna.create_study(direction="maximize", study_name=study_name)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # log summary
    row = {
        "time": now_tag(),
        "phase": "tuning",
        "config": config_name,
        "study_name": study_name,
        "best_value_f1w": study.best_value,
        "best_params_json": json.dumps(study.best_trial.params, ensure_ascii=False),
        "n_trials": n_trials,
        "tune_epochs": str(tune_epochs),
    }
    append_row_to_log(row)
    print(f"======= Tuning finished: {config_name} | best f1_weighted={study.best_value:.4f}")
    return study.best_trial.params, study.best_value

def final_train_and_save(config_name: str, best_params: dict, final_epochs=12, save_name=None):
    """
    Final training with best params (longer, ES active), save ONLY weights .pt to Drive,
    log val/test to CSV.
    """
    out_dir, best_pt = temp_paths(f"final_{config_name}")
    trainer = make_trainer(
        output_dir=out_dir,
        dropout_rate=best_params.get("dropout_rate", 0.2),
        learning_rate=best_params.get("learning_rate", 2e-5),
        weight_decay=best_params.get("weight_decay", 0.01),
        warmup_ratio=best_params.get("warmup_ratio", 0.06),
        lr_scheduler_type=best_params.get("lr_scheduler_type", "linear"),
        per_device_train_batch_size=best_params.get("per_device_train_batch_size", 32),
        per_device_eval_batch_size=64,
        num_train_epochs=final_epochs,
        gradient_accumulation_steps=best_params.get("gradient_accumulation_steps", 1),
        label_smoothing_factor=best_params.get("label_smoothing_factor", 0.0),
        fp16=True,
        report_to_wandb=False,
        run_name=f"{config_name}-final",
        best_weights_path=best_pt
    )
    trainer.train()
    val_metrics  = trainer.evaluate(ds_tok["validation"])
    test_metrics = trainer.evaluate(ds_tok["test"])

    # Rebuild and save ONLY weights to Drive
    model_for_save = BertWithDropout(
        model_name=MODEL_NAME,
        num_labels=len(ordered_labels),
        dropout_rate=best_params.get("dropout_rate", 0.2),
    )
    assert os.path.exists(best_pt), "Temp best weights not found."
    model_for_save.load_state_dict(torch.load(best_pt, map_location="cpu"))

    if save_name is None:
        save_name = f"HF_best_{config_name}_{now_tag()}.pt"
    drive_path = os.path.join(MODEL_DIR, save_name)
    torch.save(model_for_save.state_dict(), drive_path)
    print(f"========= Final best weights saved to: {drive_path}")

    # cleanup temp
    try:
        os.remove(best_pt)
    except OSError:
        pass
    del trainer, model_for_save
    torch.cuda.empty_cache(); gc.collect()

    # append final results
    row = {
        "time": now_tag(),
        "phase": "final",
        "config": config_name,
        "val_f1_weighted": float(val_metrics.get("eval_f1_weighted", 0.0)),
        "val_accuracy": float(val_metrics.get("eval_accuracy", 0.0)),
        "test_f1_weighted": float(test_metrics.get("eval_f1_weighted", 0.0)),
        "test_accuracy": float(test_metrics.get("eval_accuracy", 0.0)),
        "saved_to": drive_path,
    }
    append_row_to_log(row)
    return drive_path, val_metrics, test_metrics


# Hyperparameter selection summary:
I used a broad, well-established search space that avoids task-specific assumptions and works reliably for BERT. The tuner explores learning rate on a log scale in the classic BERT range (\~2e-5–5e-5), moderate weight decay (0.0–0.05) to preserve pretrained representations, dropout 0.10–0.30 for regularization, and a non-zero warmup ratio (\~4–12%) to stabilize early updates. I allowed common schedulers (linear / cosine / cosine\_with\_restarts / polynomial), batch sizes 16/32 with optional gradient accumulation, and short tuning epochs (e.g., 4–6) with early stopping to keep the search fast. Model selection is by macro-F1 (robust under class imbalance), with class-weighted cross-entropy to counter skewed labels; I also log weighted-F1 and accuracy for completeness. For stability, I use gradient clipping and save only the best state\_dict via a lightweight callback (no heavy HF checkpoints). In later refinement, I optionally apply discriminative learning rates (lower LR for lower encoder layers) and a slightly stronger warmup—generic, model-agnostic tweaks that often yield small but reliable gains without overfitting.


In [ ]:
# ========================================
# 11) Search spaces
#     - Stage 1: broad (fresh run)
# ========================================

# Stage 1 (broad): good for a fresh, clean run
def search_space_stage1(trial, tune_epochs):
    return {
        "dropout_rate": trial.suggest_float("dropout_rate", 0.10, 0.30),
        "learning_rate": trial.suggest_float("learning_rate", 2e-6, 5e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.00, 0.05),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.00, 0.12),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type",
                            ["linear", "cosine", "cosine_with_restarts", "polynomial"]),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.00, 0.08),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", tune_epochs[0], tune_epochs[1]),
    }

In [ ]:
# ========================================
# 12) Fresh run (Stage 1): tune → final → save weights to Drive
#     - Change N_TRIALS if you want more/less
# ========================================
N_TRIALS = 12
best_params_s1, best_val_s1 = tune_once(
    config_name="stage1_broad",
    search_space_fn=search_space_stage1,
    n_trials=N_TRIALS,
    tune_epochs=(4,6)
)

save_path_s1, val_metrics_s1, test_metrics_s1 = final_train_and_save(
    config_name="stage1_broad",
    best_params=best_params_s1,
    final_epochs=12,
    save_name="HF_best_model_stage1.pt"
)

print("Stage 1 — Validation:", val_metrics_s1)
print("Stage 1 — Test:", test_metrics_s1)


[I 2025-08-12 14:31:38,262] A new study created in memory with name: stage1_broad_20250812_143138


  0%|          | 0/12 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.793700,0.767247,0.653183,0.653183,0.663591,0.646858
2,0.514400,0.489923,0.806122,0.806122,0.812403,0.805206
3,0.378000,0.450924,0.818270,0.818270,0.823508,0.817602
4,0.272200,0.454171,0.830904,0.830904,0.836396,0.830201
5,0.218500,0.438455,0.849976,0.849976,0.854449,0.849757
6,0.218100,0.444251,0.850340,0.850340,0.854858,0.850060


...

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.469800,0.437142,0.818999,0.818999,0.825111,0.818006
2,0.337200,0.341585,0.868926,0.868926,0.873734,0.868822
3,0.167200,0.397408,0.871477,0.871477,0.875452,0.871186
4,0.086700,0.448042,0.876093,0.876093,0.879132,0.875718


[I 2025-08-12 15:47:44,779] Trial 11 finished with value: 0.8757176842038468 and parameters: {'dropout_rate': 0.11210729697348276, 'learning_rate': 4.6698588825345266e-05, 'weight_decay': 0.021806778101538145, 'warmup_ratio': 0.007822977020193009, 'lr_scheduler_type': 'linear', 'label_smoothing_factor': 0.04445989760275212, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 2, 'num_train_epochs': 4}. Best is trial 6 with value: 0.8786209676643522.
======= Tuning finished: stage1_broad | best f1_weighted=0.8786


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.576500,0.526952,0.773810,0.773810,0.780436,0.771543
2,0.417600,0.385047,0.847911,0.847911,0.851361,0.847980
3,0.304500,0.414542,0.854349,0.854349,0.857423,0.854491
4,0.237600,0.477445,0.849976,0.849976,0.854778,0.849791
5,0.187100,0.526061,0.871963,0.871963,0.875574,0.872007
6,0.120300,0.582362,0.875486,0.875486,0.878297,0.875277
7,0.146100,0.625834,0.873299,0.873299,0.876559,0.872710
8,0.107800,0.710395,0.876944,0.876944,0.879629,0.876727


Stage 2 – Hyperparameter narrowing: what we changed and why

Learning rate → 2.6e-5–4.8e-5. Stage 1’s best runs clustered around ~2.8e-5/3.9e-5/4.7e-5. Narrowing here cuts unproductive very-low/very-high LRs while keeping the proven region.

Dropout → 0.10–0.22. Top results used ~0.11–0.20. This range preserves enough regularization without over-penalizing the classifier head.

Weight decay → 0.020–0.050. Winners sat around ~0.022–0.048. This protects pretrained representations while still damping overfitting.

Warmup ratio → 0.00–0.08. Best trials included both ~0 and ~0.055; a small band keeps early training stable without slowing convergence.

Scheduler → {linear, cosine_with_restarts}. These were the most consistently strong/steady in Stage 1; we removed less reliable options to focus search.

Batch size → 16 (fixed) + GA ∈ {1,2}. Matches the strongest configs and allows adjusting effective batch size via accumulation.

Num epochs → 4–6. Aligns with where validation peaks typically appeared in Stage 1 while keeping tuning time reasonable.

Label smoothing → 0.00–0.05. Small amounts helped calibration; larger values tended to depress F1.

Everything else unchanged. We still use class-weighted CE, macro/weighted F1 for selection/reporting, gradient clipping, early stopping, and saving only the best state_dict.

In [ ]:
def search_space_stage2(trial, tune_epochs):
    return {
        "dropout_rate": trial.suggest_float("dropout_rate", 0.10, 0.22),
        "learning_rate": trial.suggest_float("learning_rate", 2.6e-5, 4.8e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.020, 0.050),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.00, 0.08),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine_with_restarts"]),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.00, 0.05),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16]),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 4, 6),
    }


In [ ]:
# ========================================
# 12) Fresh run (Stage 2): tune → final → save weights to Drive
#     - Change N_TRIALS if you want more/less
# ========================================
N_TRIALS = 12
best_params_s2, best_val_s2 = tune_once(
    config_name="stage2_broad",
    search_space_fn=search_space_stage2,
    n_trials=N_TRIALS,
    tune_epochs=(4,6)
)

save_path_s2, val_metrics_s2, test_metrics_s2 = final_train_and_save(
    config_name="stage1_broad",
    best_params=best_params_s2,
    final_epochs=12,
    save_name="HF_best_model_stage2.pt"
)

print("Stage 2 — Validation:", val_metrics_s2)
print("Stage 2 — Test:", test_metrics_s2)


**Stage 2 Results**

Validation (F1-macro): 0.8763

Test (F1-macro): 0.8589

Convergence: strong scores reached rapidly (≈ epoch 8), consistent with Stage 1’s early peaks.

**Motivation for Stage 3 — Evidence-Driven Adjustments**

The fast convergence in Stages 1–2 indicates that upper Transformer layers adapt quickly, while pushing lower layers too aggressively early can erode pretrained representations. Stage 3 therefore refines optimization mechanics to protect the backbone during the initial updates while still allowing the head/upper layers to adapt.

**Optimizer Strategy Revision — Discriminative Learning Rates (DLR)**

What changed: Introduced get_layer_id and build_param_groups to apply layer-wise LR decay (e.g., layer_decay ≈ 0.90).

Why: Lower (earlier) encoder layers receive smaller LR to preserve general linguistic features learned during pretraining; upper layers and the classifier head retain the base LR to capture task-specific signals quickly.

Mechanics: per-layer LR = base_lr * (layer_decay ** distance_from_top).

**Early-Training Stabilization — Non-Zero Warmup**

What changed: Enforced non-zero warmup by searching warmup_ratio ∈ [0.04, 0.12].

Why: Stages 1–2 already showed early performance peaks; a modest warmup tempers the first optimization steps, reducing instability and catastrophic forgetting when LR is relatively high.

**Search Space Configuration — Broad, Sane, and Grounded in Stage 2**

Learning rate: 2e-5–5e-5 (log scale) — classic BERT band that covered Stage-2 winners while trimming unproductive extremes.

Dropout: 0.10–0.30 — maintains effective regularization without over-penalizing the classifier.

Weight decay: 0.00–0.05 — protects pretrained representations while still damping overfitting.

Label smoothing: 0.00–0.08 — small values were sufficient in Stage 1–2.

Scheduler options: {linear, cosine, cosine_with_restarts, polynomial} — retained to let tuning reaffirm the stable families observed in Stage 2.

Batching: {16, 32} with GA ∈ {1, 2} — controls effective batch size without altering the data pipeline.

Epoch range: inherited from tuning bounds used in Stage 2 to keep runs efficient and comparable.

Class-weighted cross-entropy and macro-F1 remain the selection/reporting baseline to address label imbalance.

Gradient clipping and a lightweight BestWeightsSaver continue to ensure stability and minimal I/O (saving only the best state_dict).

In [ ]:
import re

def get_layer_id(name: str) -> int:
    """
    Map a parameter name to a layer id:
    - embeddings -> 0
    - encoder.layer.X -> X + 1
    - pooler / classifier head -> last layer bucket (13)
    """
    if name.startswith("bert.embeddings"):
        return 0
    m = re.search(r"bert\.encoder\.layer\.(\d+)\.", name)
    if m:
        return int(m.group(1)) + 1
    # pooler / classifier head
    return 13  # BERT-base has 12 encoder layers; 13 is the pooler/head bucket


In [ ]:
from torch.optim import AdamW

def build_param_groups(model, base_lr, weight_decay, layer_decay=0.9):
    """
    Create optimizer parameter groups with layer-wise learning-rate decay.

    Notes:
    - If layer_decay < 1.0, lower (earlier) layers learn more slowly than higher ones.
    - base_lr is applied to the top bucket; lower buckets get base_lr * layer_decay^(distance).
    """
    num_layers = 14  # ids 0..13 as returned by get_layer_id
    # Parameters excluded from weight decay
    no_decay = ["bias", "LayerNorm.weight", "LayerNorm.bias", "layer_norm.weight", "layer_norm.bias"]

    param_groups = {}
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        layer_id = get_layer_id(name)
        # Per-layer LR: base_lr * (layer_decay ** (num_layers - layer_id - 1))
        lr = base_lr * (layer_decay ** (num_layers - layer_id - 1))
        use_decay = not any(nd in name for nd in no_decay)
        key = (layer_id, use_decay)
        if key not in param_groups:
            param_groups[key] = {
                "params": [],
                "lr": lr,
                "weight_decay": (weight_decay if use_decay else 0.0),
            }
        param_groups[key]["params"].append(param)
    return list(param_groups.values())

In [ ]:
# ========================================
# 9) Trainer factory (no HF checkpoints) + BestWeightsSaver callback
#    - Discriminative LRs (layer-wise decay) + explicit scheduler
#    - Saves ONLY best weights (state_dict) to a temp .pt
#    - EarlyStopping uses eval_f1_weighted from compute_metrics_fn
# ========================================
import os
import gc
import re
import torch
import types
from datetime import datetime
from torch.optim import AdamW
from transformers import (
    TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback, get_scheduler
)

class BestWeightsSaver(TrainerCallback):
    """
    When 'eval_f1_weighted' improves, save ONLY model.state_dict() to `best_weights_path`.
    """
    def __init__(self, best_weights_path: str, metric_name: str = "eval_f1_weighted"):
        self.best_weights_path = best_weights_path
        self.metric_name = metric_name
        self.best_score = None
        self._trainer = None  # injected on attach

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if not metrics or self.metric_name not in metrics:
            return
        score = float(metrics[self.metric_name])
        if (self.best_score is None) or (score > self.best_score):
            self.best_score = score
            torch.save(self._trainer.model.state_dict(), self.best_weights_path)

MODEL_NAME = "bert-base-uncased"

# ---------- Helpers for discriminative learning rates ----------
def get_layer_id(name: str) -> int:
    """
    Map parameter name to a layer id:
      - embeddings -> 0
      - encoder.layer.X -> X+1  (X in [0..11] for BERT-base) => 1..12
      - pooler / classifier head -> 13
    """
    if name.startswith("bert.embeddings"):
        return 0
    m = re.search(r"bert\.encoder\.layer\.(\d+)\.", name)
    if m:
        return int(m.group(1)) + 1
    return 13  # pooler/head

def build_param_groups(model, base_lr: float, weight_decay: float, layer_decay: float = 0.90):
    """
    Create optimizer param groups with layer-wise LR decay.
    Lower layers get smaller LR; higher layers get larger LR.
    """
    no_decay = ["bias", "LayerNorm.weight", "LayerNorm.bias", "layer_norm.weight", "layer_norm.bias"]
    num_layers = 14  # ids: 0..13
    buckets = {}
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        lid = get_layer_id(name)
        lr = base_lr * (layer_decay ** (num_layers - lid - 1))
        use_decay = not any(nd in name for nd in no_decay)
        key = (lid, use_decay)
        if key not in buckets:
            buckets[key] = {
                "params": [],
                "lr": lr,
                "weight_decay": (weight_decay if use_decay else 0.0),
            }
        buckets[key]["params"].append(param)
    return list(buckets.values())

def make_trainer(
    output_dir,
    dropout_rate=0.2,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.08,                  # stronger warmup by default
    lr_scheduler_type="linear",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    gradient_accumulation_steps=1,
    label_smoothing_factor=0.0,
    fp16=True,
    report_to_wandb=False,
    run_name="trial",
    best_weights_path="/tmp/best_weights.pt",
    layer_decay=0.90,                   # LR decay factor across layers (0.85–0.95 typical)
):
    # Build model with the custom head
    model = BertWithDropout(
        model_name=MODEL_NAME,
        num_labels=len(ordered_labels),
        dropout_rate=dropout_rate,
    )

    # TrainingArguments: keep HF checkpoints off; early stopping driven by metrics
    args = TrainingArguments(
        output_dir=output_dir,                 # keep under /tmp to avoid Drive writes
        eval_strategy="epoch",
        save_strategy="no",                    # no HF checkpoints
        load_best_model_at_end=False,          # best is handled by BestWeightsSaver
        metric_for_best_model="f1_weighted",
        greater_is_better=True,
        logging_strategy="steps",
        logging_steps=50,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        num_train_epochs=num_train_epochs,
        warmup_ratio=0.0,                      # overridden by explicit scheduler below
        learning_rate=learning_rate,           # base LR for the head/top layer
        weight_decay=weight_decay,
        lr_scheduler_type="linear",            # unused by Trainer since we pass our own scheduler
        gradient_accumulation_steps=gradient_accumulation_steps,
        label_smoothing_factor=label_smoothing_factor,
        max_grad_norm=0.5,                     # gradient clipping
        fp16=fp16,
        report_to=(["wandb"] if report_to_wandb else ["none"]),
        run_name=run_name,
        seed=42,
        dataloader_num_workers=2,
    )

    # Build discriminative LR optimizer
    param_groups = build_param_groups(
        model, base_lr=learning_rate, weight_decay=weight_decay, layer_decay=layer_decay
    )
    optimizer = AdamW(param_groups, betas=(0.9, 0.999), eps=1e-8)

    # Compute total training steps and warmup steps for explicit scheduler
    # Note: effective batch size accounts for gradient accumulation
    effective_bs = per_device_train_batch_size * max(1, gradient_accumulation_steps)
    num_update_steps_per_epoch = max(1, len(ds_tok["train"]) // effective_bs)
    num_training_steps = int(num_update_steps_per_epoch * num_train_epochs)
    num_warmup_steps = int(warmup_ratio * num_training_steps)

    lr_scheduler = get_scheduler(
        name=lr_scheduler_type,                # "linear", "cosine_with_restarts", etc.
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
    )

    # Build Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tok["train"],
        eval_dataset=ds_tok["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_fn,
        optimizers=(optimizer, lr_scheduler),  # <-- pass our optimizer & scheduler
    )

    # Custom weighted CE loss (device-aware)
    trainer.compute_loss = types.MethodType(custom_compute_loss, trainer)

    # Early stopping (small patience keeps it agile)
    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

    # Attach best-weights saver
    best_cb = BestWeightsSaver(best_weights_path=best_weights_path, metric_name="eval_f1_weighted")
    best_cb._trainer = trainer
    trainer.add_callback(best_cb)

    return trainer


In [ ]:
# ========================================
# 10) Generic experiment runner (replaces your old "part 10")
#     - tune_once(): Optuna tuning for a given search space
#     - final_train_and_save(): long-ish final run + save ONLY weights to Drive
# ========================================
import json
import optuna
import pandas as pd

MODEL_DIR  =data_dir
EXPERIMENTS_LOG = os.path.join(MODEL_DIR, "HF_experiments_log_3b.csv")  # append-only CSV

def now_tag():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def temp_paths(tag: str):
    stamp = now_tag()
    out_dir = f"/tmp/{tag}_{stamp}"
    best_pt = f"/tmp/{tag}_{stamp}_best.pt"
    os.makedirs(out_dir, exist_ok=True)
    return out_dir, best_pt

def append_row_to_log(row: dict):
    if os.path.exists(EXPERIMENTS_LOG):
        df = pd.read_csv(EXPERIMENTS_LOG)
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    else:
        df = pd.DataFrame([row])
    df.to_csv(EXPERIMENTS_LOG, index=False)

def tune_once(config_name: str, search_space_fn, n_trials=10, tune_epochs=(3,6)):
    """
    Run Optuna on a provided search space function.
    Returns (best_params, best_value).
    """
    def objective(trial: optuna.trial.Trial):
        hp = search_space_fn(trial, tune_epochs)
        out_dir, best_pt = temp_paths(f"tune_{config_name}_t{trial.number}")
        trainer = make_trainer(
            output_dir=out_dir,
            dropout_rate=hp["dropout_rate"],
            learning_rate=hp["learning_rate"],
            weight_decay=hp["weight_decay"],
            warmup_ratio=hp["warmup_ratio"],
            lr_scheduler_type=hp["lr_scheduler_type"],
            per_device_train_batch_size=hp["per_device_train_batch_size"],
            per_device_eval_batch_size=64,
            num_train_epochs=hp["num_train_epochs"],
            gradient_accumulation_steps=hp["gradient_accumulation_steps"],
            label_smoothing_factor=hp["label_smoothing_factor"],
            layer_decay=hp["layer_decay"],
            fp16=True,
            report_to_wandb=False,
            run_name=f"{config_name}-trial-{trial.number}",
            best_weights_path=best_pt
        )
        trainer.train()
        metrics = trainer.evaluate(ds_tok["validation"])
        score = float(metrics.get("eval_f1_weighted") or metrics.get("f1_weighted") or 0.0)

        # Clean temp
        try:
            if os.path.exists(best_pt):
                os.remove(best_pt)
        except OSError:
            pass
        del trainer
        torch.cuda.empty_cache(); gc.collect()
        return score

    study_name = f"{config_name}_{now_tag()}"
    study = optuna.create_study(direction="maximize", study_name=study_name)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # log summary
    row = {
        "time": now_tag(),
        "phase": "tuning",
        "config": config_name,
        "study_name": study_name,
        "best_value_f1w": study.best_value,
        "best_params_json": json.dumps(study.best_trial.params, ensure_ascii=False),
        "n_trials": n_trials,
        "tune_epochs": str(tune_epochs),
    }
    append_row_to_log(row)
    print(f"======= Tuning finished: {config_name} | best f1_weighted={study.best_value:.4f}")
    return study.best_trial.params, study.best_value

def final_train_and_save(config_name: str, best_params: dict, final_epochs=12, save_name=None):
    """
    Final training with best params (longer, ES active), save ONLY weights .pt to Drive,
    log val/test to CSV.
    """
    out_dir, best_pt = temp_paths(f"final_{config_name}")
    trainer = make_trainer(
        output_dir=out_dir,
        dropout_rate=best_params.get("dropout_rate", 0.2),
        learning_rate=best_params.get("learning_rate", 2e-5),
        weight_decay=best_params.get("weight_decay", 0.01),
        warmup_ratio=best_params.get("warmup_ratio", 0.06),
        lr_scheduler_type=best_params.get("lr_scheduler_type", "linear"),
        per_device_train_batch_size=best_params.get("per_device_train_batch_size", 32),
        per_device_eval_batch_size=64,
        num_train_epochs=final_epochs,
        gradient_accumulation_steps=best_params.get("gradient_accumulation_steps", 1),
        label_smoothing_factor=best_params.get("label_smoothing_factor", 0.0),
        fp16=True,
        report_to_wandb=False,
        run_name=f"{config_name}-final",
        best_weights_path=best_pt
    )
    trainer.train()
    val_metrics  = trainer.evaluate(ds_tok["validation"])
    test_metrics = trainer.evaluate(ds_tok["test"])

    # Rebuild and save ONLY weights to Drive
    model_for_save = BertWithDropout(
        model_name=MODEL_NAME,
        num_labels=len(ordered_labels),
        dropout_rate=best_params.get("dropout_rate", 0.2),
    )
    assert os.path.exists(best_pt), "Temp best weights not found."
    model_for_save.load_state_dict(torch.load(best_pt, map_location="cpu"))

    if save_name is None:
        save_name = f"HF_best_{config_name}_{now_tag()}.pt"
    drive_path = os.path.join(MODEL_DIR, save_name)
    torch.save(model_for_save.state_dict(), drive_path)
    print(f"========= Final best weights saved to: {drive_path}")

    # cleanup temp
    try:
        os.remove(best_pt)
    except OSError:
        pass
    del trainer, model_for_save
    torch.cuda.empty_cache(); gc.collect()

    # append final results
    row = {
        "time": now_tag(),
        "phase": "final",
        "config": config_name,
        "val_f1_weighted": float(val_metrics.get("eval_f1_weighted", 0.0)),
        "val_accuracy": float(val_metrics.get("eval_accuracy", 0.0)),
        "test_f1_weighted": float(test_metrics.get("eval_f1_weighted", 0.0)),
        "test_accuracy": float(test_metrics.get("eval_accuracy", 0.0)),
        "saved_to": drive_path,
    }
    append_row_to_log(row)
    return drive_path, val_metrics, test_metrics


In [ ]:
def search_space_stage3(trial, tune_epochs):
    return {
        "dropout_rate": trial.suggest_float("dropout_rate", 0.10, 0.30),
        "learning_rate": trial.suggest_float("learning_rate", 2e-5, 5e-5, log=True),   # narrower & effective
        "weight_decay": trial.suggest_float("weight_decay", 0.00, 0.05),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.04, 0.12),               # no zero-warmup
        "lr_scheduler_type": trial.suggest_categorical(
            "lr_scheduler_type", ["linear", "cosine", "cosine_with_restarts", "polynomial"]
        ),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.00, 0.08),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", tune_epochs[0], tune_epochs[1]),
    }


In [ ]:
# ========================================
# 12) Fresh run (Stage 3): tune → final → save weights to Drive
#     - Change N_TRIALS if you want more/less
# ========================================
N_TRIALS = 12
best_params_s3, best_val_s3 = tune_once(
    config_name="stage3_broad",
    search_space_fn=search_space_stage3,
    n_trials=N_TRIALS,
    tune_epochs=(4,6)
)

save_path_s3, val_metrics_s3, test_metrics_s3 = final_train_and_save(
    config_name="stage3_broad",
    best_params=best_params_s3,
    final_epochs=12,
    save_name="HF_best_model_stage3.pt"
)

print("Stage 3 — Validation:", val_metrics_s3)
print("Stage 3 — Test:", test_metrics_s3)


[I 2025-08-13 08:56:41,150] A new study created in memory with name: stage3_broad_20250813_085641


  0%|          | 0/12 [00:00<?, ?it/s]

/tmp/ipython-input-1330159245.py:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.723600,0.614806,0.751822,0.751822,0.760999,0.750810
2,0.408300,0.441492,0.826895,0.826895,0.833389,0.826286
3,0.297800,0.400215,0.847303,0.847303,0.852721,0.847300
4,0.221100,0.415122,0.851798,0.851798,0.856496,0.851328
5,0.153800,0.439108,0.856778,0.856778,0.860399,0.856263
6,0.120200,0.436310,0.860787,0.860787,0.864290,0.860402


[I 2025-08-13 09:01:31,841] Trial 0 finished with value: 0.8604021607335369 and parameters: {'dropout_rate': 0.15595333678907414, 'learning_rate': 2.1701780299161078e-05, 'weight_decay': 0.044137549166471716, 'warmup_ratio': 0.10265306726055529, 'lr_scheduler_type': 'cosine_with_restarts', 'label_smoothing_factor': 0.05363132769093291, 'per_device_train_batch_size': 32, 'gradient_accumulation_steps': 2, 'num_train_epochs': 6}. Best is trial 0 with value: 0.8604021607335369.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.520800,0.470885,0.801749,0.801749,0.808603,0.800482
2,0.353300,0.431047,0.848397,0.848397,0.851065,0.848306
3,0.187900,0.478019,0.874271,0.874271,0.877952,0.874094
4,0.135200,0.549288,0.881560,0.881560,0.884313,0.881375


[I 2025-08-13 09:58:02,459] Trial 11 finished with value: 0.8813748459092184 and parameters: {'dropout_rate': 0.29529657837750006, 'learning_rate': 3.6074197197478196e-05, 'weight_decay': 0.013861334169702078, 'warmup_ratio': 0.08243887040342272, 'lr_scheduler_type': 'cosine', 'label_smoothing_factor': 0.07747124325253954, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 1, 'num_train_epochs': 4}. Best is trial 11 with value: 0.8813748459092184.
======= Tuning finished: stage3_broad | best f1_weighted=0.8814


/tmp/ipython-input-1330159245.py:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.583900,0.600934,0.737366,0.737366,0.744489,0.734013
2,0.434000,0.455987,0.834913,0.834913,0.838954,0.835694
3,0.304900,0.411162,0.856050,0.856050,0.860470,0.855977
4,0.246600,0.531000,0.828596,0.828596,0.836596,0.828459
5,0.196500,0.708884,0.857264,0.857264,0.859665,0.856840
6,0.169900,0.563529,0.871963,0.871963,0.875413,0.871656
7,0.102800,0.674311,0.882046,0.882046,0.884167,0.881976
8,0.084400,0.757486,0.882532,0.882532,0.884681,0.882452
9,0.026900,0.766624,0.882532,0.882532,0.884833,0.882137
10,0.028100,0.855928,0.884475,0.884475,0.887198,0.884369


========= Final best weights saved to: /content/drive/MyDrive/deep_learning/HF_best_model_stage3.pt
Stage 3 — Validation: {'eval_loss': 0.8826243281364441, 'eval_accuracy': 0.8852040816326531, 'eval_f1_micro': 0.8852040816326531, 'eval_f1_macro': 0.8877186499037657, 'eval_f1_weighted': 0.885104591068076, 'eval_runtime': 2.3346, 'eval_samples_per_second': 3526.098, 'eval_steps_per_second': 55.256, 'epoch': 12.0}
Stage 3 — Test: {'eval_loss': 1.0449219942092896, 'eval_accuracy': 0.8622959452343338, 'eval_f1_micro': 0.8622959452343338, 'eval_f1_macro': 0.8643927270667865, 'eval_f1_weighted': 0.8626914491589446, 'eval_runtime': 1.2844, 'eval_samples_per_second': 2957.044, 'eval_steps_per_second': 46.715, 'epoch': 12.0}


**Stage 3 results**.
 After introducing Discriminative Learning Rates (layer-wise LR decay) and enforcing a non-zero warmup, performance improved modestly yet consistently: val F1-macro = 0.8877 and test F1-macro = 0.8644 (vs. Stage 2: 0.8763 / 0.8589). This aligns with the observation from Stages 1–2 that strong scores arrive early: protecting lower encoder layers while letting upper layers/head adapt faster yields slightly better generalization.

Why Stage 3b. Based on Stage 3, we narrowed the search to the regime that worked best and exposed the DLR strength as a tunable hyperparameter:

Learning rate 3.2e-5–4.2e-5 and dropout 0.14–0.22: centered around Stage-3 winners, balancing capacity and regularization.

Weight decay 0.025–0.040: enough to curb overfitting without eroding pretrained features.

Warmup ratio 0.06–0.10 with cosine_with_restarts: stabilizes the sensitive early steps under higher LR and supports smooth restarts.

Label smoothing 0.01–0.04: mild calibration without depressing F1.

Batch size 16, GA=1; epochs 6–9: practical budget that still gives sufficient post-warmup updates.

layer_decay 0.88–0.94 (new): explicitly tunes how aggressively lower layers are slowed relative to the head, letting the search find the best preservation/adaptation trade-off for this dataset.

In [ ]:
def search_space_stage3b(trial, tune_epochs):
    return {
        "dropout_rate": trial.suggest_float("dropout_rate", 0.14, 0.22),
        "learning_rate": trial.suggest_float("learning_rate", 3.2e-5, 4.2e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.025, 0.040),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.06, 0.10),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["cosine_with_restarts"]),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.01, 0.04),
        "per_device_train_batch_size": 16,
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 6, 9),
        "layer_decay": trial.suggest_float("layer_decay", 0.88, 0.94),   # <— new
    }


In [ ]:
# ========================================
# 12) Fresh run (Stage 3b): tune → final → save weights to Drive
#     - Change N_TRIALS if you want more/less
# ========================================
N_TRIALS = 12
best_params_s3b, best_val_s3b = tune_once(
    config_name="stage3_broad",
    search_space_fn=search_space_stage3b,
    n_trials=N_TRIALS,
    tune_epochs=(4,6)
)

save_path_s3b, val_metrics_s3, test_metrics_s3b = final_train_and_save(
    config_name="stage3b_broad",
    best_params=best_params_s3b,
    final_epochs=12,
    save_name="HF_best_model_stage3b.pt"
)

print("Stage 3b — Validation:", val_metrics_s3)
print("Stage 3b — Test:", test_metrics_s3b)

[I 2025-08-16 08:31:15,222] A new study created in memory with name: stage3_broad_20250816_083115


  0%|          | 0/12 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/tmp/ipython-input-1330159245.py:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.542000,0.532396,0.779519,0.779519,0.785335,0.777809
2,0.443800,0.396256,0.858601,0.858601,0.862638,0.858918
3,0.287000,0.377104,0.871477,0.871477,0.875110,0.871215
4,0.214000,0.512250,0.861516,0.861516,0.865888,0.861264
5,0.221600,0.602413,0.871842,0.871842,0.875095,0.871774
6,0.151300,0.647518,0.884232,0.884232,0.886910,0.884285
7,0.077500,0.756742,0.873056,0.873056,0.875172,0.872377
8,0.053000,0.786963,0.883989,0.883989,0.886705,0.883990


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.565700,0.651515,0.715743,0.715743,0.722870,0.710955
2,0.429900,0.432665,0.847425,0.847425,0.851879,0.847775
3,0.292700,0.409695,0.862002,0.862002,0.865143,0.862364
4,0.205100,0.517238,0.850826,0.850826,0.855060,0.850528
5,0.179400,0.599170,0.866983,0.866983,0.870638,0.866610
6,0.123900,0.648700,0.883989,0.883989,0.886566,0.883885
7,0.093200,0.697095,0.879616,0.879616,0.882712,0.879159
8,0.033800,0.771816,0.885326,0.885326,0.888181,0.885073
9,0.027900,0.790443,0.887026,0.887026,0.889879,0.886891


[I 2025-08-16 10:55:50,182] Trial 11 finished with value: 0.8868905874305938 and parameters: {'dropout_rate': 0.14022414018100807, 'learning_rate': 3.424734557815674e-05, 'weight_decay': 0.03852529221309016, 'warmup_ratio': 0.09914937113184212, 'lr_scheduler_type': 'cosine_with_restarts', 'label_smoothing_factor': 0.036533919355799964, 'gradient_accumulation_steps': 1, 'num_train_epochs': 9, 'layer_decay': 0.9385501488469521}. Best is trial 11 with value: 0.8868905874305938.
======= Tuning finished: stage3_broad | best f1_weighted=0.8869


/tmp/ipython-input-1330159245.py:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.656700,0.608345,0.740525,0.740525,0.749467,0.737750
2,0.434100,0.402128,0.844145,0.844145,0.848885,0.844039
3,0.322500,0.379350,0.866254,0.866254,0.869690,0.866641
4,0.193600,0.390666,0.853741,0.853741,0.856611,0.853582
5,0.164600,0.498355,0.860058,0.860058,0.864477,0.859938


========= Final best weights saved to: /content/drive/MyDrive/deep_learning/HF_best_model_stage3b.pt
Stage 3b — Validation: {'eval_loss': 0.4983547329902649, 'eval_accuracy': 0.8600583090379009, 'eval_f1_micro': 0.8600583090379009, 'eval_f1_macro': 0.864477472517444, 'eval_f1_weighted': 0.8599380727527254, 'eval_runtime': 2.3104, 'eval_samples_per_second': 3563.037, 'eval_steps_per_second': 55.835, 'epoch': 5.0}
Stage 3b — Test: {'eval_loss': 0.5515292286872864, 'eval_accuracy': 0.8430753027909426, 'eval_f1_micro': 0.8430753027909426, 'eval_f1_macro': 0.8474695537131105, 'eval_f1_weighted': 0.8429512336449717, 'eval_runtime': 1.2776, 'eval_samples_per_second': 2972.719, 'eval_steps_per_second': 46.962, 'epoch': 5.0}


**Stage 3b underperformed relative to earlier stages**, so we stopped the search. With DLR + non-zero warmup in Stage 3 we reached val F1-macro 0.8877 and test F1-macro 0.8644, improving on Stage 2 (0.8763 / 0.8589). In contrast, Stage 3b—run with a tighter LR band, higher warmup, moderate dropout/weight-decay, and only ~5 epochs—regressed to val 0.8645 and test 0.8475. The likely causes are:


1.  over-regularization (dropout + WD + label smoothing)
2.  too much warmup for a short training budget (fewer effective post-warmup updates)
3. a layer-decay that slowed lower layers more than needed under this budget.

Given diminishing returns and limited time, we stopped after Stage 3b and selected the Stage 3 configuration as the final model. (If we had the time , we would re-train the Stage 3 best with 2–3 seeds for robustness and optionally retune layer_decay/warmup with a longer schedule.)